In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.4 MB/s 
     |████████████████████████████████| 7.6 MB 59.8 MB/s 
     |████████████████████████████████| 182 kB 51.2 MB/s 


In [ ]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from summarizer import Summarizer
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = Summarizer()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def checkParent(tag):
  while True: 
    try:
      up = tag.parent
      if up.name == "tr":
        return up 
      else:
        tag = up
    except AttributeError:
      return False

In [ ]:
import re
from bs4 import BeautifulSoup
def getInternalUrl(filePath):
  internalDict = {}
  pattern = re.compile("^#")
  soup = BeautifulSoup(open(filePath), 'html.parser')
  for a in soup.find_all('a', href=pattern):
    # print(a)
    if re.match(r'^([\s\d]+)$', a.text):
      continue

    up = checkParent(a)
    # print("item" in up.text.lower())
    if up != False:
      if a['href'] not in internalDict.keys() and "item" in up.text.lower():
        internalDict[a['href']] = up.text
        # print("Found the URL: " +  a['href'] + " " + up.text)
  return soup, internalDict

In [ ]:
def getItemDict(internalDict):  
  nameItem = ["Business","Risk Factors","Legal Proceedings","Management’s Discussion ", "Quantitative and Qualitative","Financial Statements and Supp"]
  ItemDict = {}
  for key,value in internalDict.items():
    for el in nameItem:
      if el.lower() in value.lower():
        ItemDict[key] = value
      
  ItemID = ["item1","item1a","item3","item7","item7a","item8"]
  if len(ItemDict.keys()) != 6:
    ItemDict = {}
    for key,value in internalDict.items() :
      for el in ItemID:
        if el in value.replace(" ","").lower() and re.search("{}\.".format(el),value.replace(" ","").lower()):
          ItemDict[key] = value
  return ItemDict

In [ ]:
def GetItemIndex(ItemDict,internalDict):
  ItemList = list(ItemDict.keys())
  internalList = list(internalDict.keys())
  ItemListIndex = []
  for el in ItemList:
    ItemListIndex.append(internalList.index(el))
  return internalList, ItemListIndex

In [ ]:
def getBelow(start):
  textList = []
  div_start = start
  while True:

    while True:
      if start == None:
        break
      textList.append(start)
      next = start.nextSibling
      start = next

    if len(textList) < 5:
      textList = []
      start = div_start.parent
      div_start = start
    else:
      return textList


  return textList

In [ ]:
def SummerizeAndAnalyze(textjoined):
  result = model(textjoined, min_length=20,max_length=200)
  summary = "".join(result)
  try:
    result = model(textjoined, min_length=20,max_length=200)
    summary = "".join(result)
    inputs = tokenizer(summary, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
  except RuntimeError:
    result = model(textjoined, min_length=20,max_length=90)
    summary = "".join(result)
    inputs = tokenizer(summary, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
  return outputs

In [ ]:
import json
import numpy as np

def Process10K(filePath,ticker):
  soup, internalDict = getInternalUrl(filePath)
  
  ItemDict = getItemDict(internalDict)
  if len(ItemDict) != 6 : return "!=6"



  internalList, ItemListIndex = GetItemIndex(ItemDict,internalDict)
  saveDict = {}
  for startIndex in ItemListIndex:

    try:
      StartUrl = internalList[startIndex].replace("#","")
      EndUrl = internalList[startIndex+1].replace("#","")
    except:
      return "StartUrl"

    if soup.find("div",{"id":StartUrl}) != None:
      start = soup.find("div",{"id":StartUrl})
      end = soup.find("div",{"id":EndUrl})
    else:
      start = soup.find("a",{"name":StartUrl})
      end = soup.find("a",{"name":EndUrl})
    
    if start == None : return "start == None"
    if end == None : return "end == None"
    
    textList = []
    for item in getBelow(start):
      if item not in getBelow(end):
        try:
          textList.append(item.text)
        except:
          pass
    
    textList = [item for item in textList if len(item) > 50]

    chunks = np.array_split(textList, 4)
    textjoined = ' '.join(textList)

    
    
    SentimentArray = []
    for chunk in chunks:
      textjoined = ' '.join(chunk)
      SentimentArray.append(SummerizeAndAnalyze(textjoined).tolist()[0])

    itemName = ItemDict[internalList[startIndex]]
    saveDict[itemName] = SentimentArray


  with open("/content/drive/MyDrive/10K/10K {}/10K Processed/{}.json".format(year,ticker), "w") as write_file:
    json.dump(saveDict, write_file, indent=4)
  return "success"

In [ ]:
import os
import pickle
year = 2016
worker = 2
with open ('/content/drive/MyDrive/10K/10K {}/meta/meta{}.pkl'.format(year,worker), 'rb') as fp:
    tickerlist = pickle.load(fp)

processed =  os.listdir("/content/drive/MyDrive/10K/10K {}/10K Processed".format(year))
processed = [el.replace(".json","") for el in processed]

remain = []
for el in tickerlist:
  if el not in processed:
    remain.append(el)
  # remain.remove('.ipynb_checkpoints')

for ticker in remain:
  path = "/content/drive/MyDrive/10K/10K {}/sec-edgar-filings/{}/10-K/".format(year,ticker)
  folder = os.listdir(path)[0]
  filePath = path + folder
  for file in os.listdir(path + folder):
      if file.endswith(".html"):
          filePath = os.path.join(filePath, file)
            
  print(Process10K(filePath,ticker) , ticker)